In [19]:
import pandas as pd

df = pd.read_csv('NU_Courses\semester_6\MI\jaundice_detector_ml\ml_model\jaundice_dataset\chd_jaundice_published_2.csv')





SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: malformed \N character escape (2216216134.py, line 3)

In [2]:
df.head()

,patient_id,image_idx,gender,gestational_age,age(day),weight,blood(mg/dL),Treatment
0,3,0003-1.jpg,F,40,5.2,3280,3.9,0
1,3,0003-2.jpg,F,40,5.2,3280,3.9,0
2,3,0003-3.jpg,F,40,5.2,3280,3.9,0
3,35,0035-1.jpg,M,39,8.7,3760,12.2,0
4,35,0035-2.jpg,M,39,8.7,3760,12.2,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2235 entries, 0 to 2234
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   patient_id       2235 non-null   int64  
 1   image_idx        2235 non-null   object 
 2   gender           2235 non-null   object 
 3   gestational_age  2235 non-null   int64  
 4   age(day)         2229 non-null   float64
 5   weight           2235 non-null   int64  
 6   blood(mg/dL)     2235 non-null   float64
 7   Treatment        2235 non-null   int64  
dtypes: float64(2), int64(4), object(2)
memory usage: 139.8+ KB


In [4]:
df.describe()

,patient_id,gestational_age,age(day),weight,blood(mg/dL),Treatment
count,2235.000000,2235.000000,2229.000000,2235.000000,2235.000000,2235.000000
mean,747.554362,38.293960,6.599192,3158.778523,11.206846,0.417450
std,420.653073,1.807409,5.703538,529.903610,5.213196,0.493249
min,1.000000,30.000000,0.100000,1690.000000,0.000000,0.000000
25%,356.000000,37.000000,2.500000,2800.000000,7.400000,0.000000
50%,794.000000,39.000000,5.000000,3180.000000,11.800000,0.000000
75%,1139.000000,40.000000,9.000000,3500.000000,15.100000,1.000000
max,1351.000000,42.000000,34.000000,5290.000000,25.700000,1.000000


In [5]:
df.duplicated()

0       False
1       False
2       False
3       False
4       False
        ...  
2230    False
2231    False
2232    False
2233    False
2234    False
Length: 2235, dtype: bool

In [6]:
df.isnull().sum()


patient_id         0
image_idx          0
gender             0
gestational_age    0
age(day)           6
weight             0
blood(mg/dL)       0
Treatment          0
dtype: int64

In [7]:
df.isnull().any(axis=0)

patient_id         False
image_idx          False
gender             False
gestational_age    False
age(day)            True
weight             False
blood(mg/dL)       False
Treatment          False
dtype: bool

In [8]:
df.columns

Index(['patient_id', 'image_idx', 'gender', 'gestational_age', 'age(day)',
       'weight', 'blood(mg/dL)', 'Treatment'],
      dtype='object')

In [9]:
df = df.drop(['gender', 'age(day)', 'weight'], axis=1, errors='ignore')
df.to_csv('chd_jaundice_published_2.csv', index=False)


In [10]:
df["Treatment"] = df["Treatment"].map({0: "not_jaundiced", 1: "jaundiced"})
df.to_csv('chd_jaundice_published_2.csv', index=False)

In [11]:


import tensorflow as tf
import os
import numpy as np

localDatasetPath = "images"

image_names = df['image_idx'].tolist()

processed_images = []

def preprocess_image_tf(image_id):
    filename = f"{image_id}" 
    full_path = os.path.join(localDatasetPath, filename)

    img = tf.io.read_file(full_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, [224, 224])
    img = tf.cast(img, tf.float32) / 255.0
    return img

for image_id in image_names:
    try:
        processed_img = preprocess_image_tf(image_id)
        processed_images.append(processed_img.numpy())
    except Exception as e:
        print(f"❌ Error processing {image_id}: {e}")

processed_images = np.array(processed_images)
print("✅ Final shape:", processed_images.shape)

✅ Final shape: (2235, 224, 224, 3)


In [12]:
from sklearn.model_selection import train_test_split
import numpy as np


X = processed_images  
y = df["Treatment"].map({"not_jaundiced": 0, "jaundiced": 1}).values  


X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42)

print(" Shapes:")
print("Train:", X_train.shape, y_train.shape)
print("Validation:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)


 Shapes:
Train: (1564, 224, 224, 3) (1564,)
Validation: (335, 224, 224, 3) (335,)
Test: (336, 224, 224, 3) (336,)


In [13]:

import tensorflow as tf

# Your images are already preprocessed and split as X_train, X_val, X_test, y_train, y_val, y_test

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(32).prefetch(tf.data.AUTOTUNE)


In [15]:
import os
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models

# Remove corrupted EfficientNetB0 weights if they exist
keras_cache_dir = os.path.expanduser("~/.keras/models")
for fname in os.listdir(keras_cache_dir):
    if "efficientnetb0" in fname.lower():
        os.remove(os.path.join(keras_cache_dir, fname))

base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze for transfer learning

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


16705208/16705208 [==============================] - 4s 0us/step


In [16]:
history = model.fit(train_ds, validation_data=val_ds, epochs=10)


Epoch 1/10
49/49 [==============================] - 55s 1s/step - loss: 0.6890 - accuracy: 0.5659 - val_loss: 0.6796 - val_accuracy: 0.5821
Epoch 2/10
49/49 [==============================] - 47s 967ms/step - loss: 0.6883 - accuracy: 0.5678 - val_loss: 0.6796 - val_accuracy: 0.5821
Epoch 3/10
49/49 [==============================] - 46s 950ms/step - loss: 0.6840 - accuracy: 0.5710 - val_loss: 0.6796 - val_accuracy: 0.5821
Epoch 4/10
49/49 [==============================] - 46s 945ms/step - loss: 0.6868 - accuracy: 0.5684 - val_loss: 0.6795 - val_accuracy: 0.5821
Epoch 5/10
49/49 [==============================] - 46s 940ms/step - loss: 0.6859 - accuracy: 0.5780 - val_loss: 0.6795 - val_accuracy: 0.5821
Epoch 6/10
49/49 [==============================] - 46s 939ms/step - loss: 0.6855 - accuracy: 0.5793 - val_loss: 0.6795 - val_accuracy: 0.5821
Epoch 7/10
49/49 [==============================] - 46s 939ms/step - loss: 0.6851 - accuracy: 0.5697 - val_loss: 0.6795 - val_accuracy: 0.5821
Ep

In [ ]:
# model.save("efficientnet_jaundice_model.h5")

TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.

In [19]:
base_model.trainable = True

# Optional: Freeze first few layers to avoid catastrophic forgetting
for layer in base_model.layers[:100]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=5)

Epoch 1/5
49/49 [==============================] - 100s 2s/step - loss: 0.7077 - accuracy: 0.5000 - val_loss: 0.6825 - val_accuracy: 0.5821
Epoch 2/5
49/49 [==============================] - 94s 2s/step - loss: 0.6914 - accuracy: 0.5294 - val_loss: 0.6925 - val_accuracy: 0.5582
Epoch 3/5
49/49 [==============================] - 91s 2s/step - loss: 0.6870 - accuracy: 0.5480 - val_loss: 0.7039 - val_accuracy: 0.4179
Epoch 4/5
49/49 [==============================] - 86s 2s/step - loss: 0.6755 - accuracy: 0.5812 - val_loss: 0.7111 - val_accuracy: 0.4179
Epoch 5/5
49/49 [==============================] - 88s 2s/step - loss: 0.6692 - accuracy: 0.5959 - val_loss: 0.7314 - val_accuracy: 0.4209
